In [1]:
import numpy as np
import pandas as pd

import sys
import os
import random

from PIL import Image

import cv2

import matplotlib.pyplot as plt

# 1. Add an alpha channel to the image

In [2]:
def add_alpha_convert_to_png(path):
    """
    adds an alpha channel to the image
    converts it to png
    turns all pixels with value higher than 200 to white
    """  
    for root, dirs, files in os.walk(path):
        files = [f for f in files if not f[0] == '.' and f.find('bg') == -1] 
        for file in files: 
            img_path = os.path.join(root, file)
            img = Image.open(img_path)
            img = img.convert("RGBA")
            data = img.getdata()
            
            newData = []
            for item in data:
                if item[0] > 200 and item[1] > 200 and item[2] > 200:
                    newData.append((255, 255, 255, 0))
                else:
                    newData.append(item)

            img.putdata(newData)
            file_name = os.path.basename(img_path).split('.')[0]
            file_png_path = os.path.join(root, f'{file_name}.png')
            img.save(file_png_path, "PNG")

# 2. Load random background

In [3]:
def load_bg_random_art(bg_path):
    """
    chooses a random background image
    """    
    bg_folder = bg_path
    bg_files = os.listdir(bg_folder)
    bg_files = [f for f in bg_files if not f[0] == '.']
    bg_index = random.randrange(0, len(bg_files))
    
    bg = os.path.join(bg_path, bg_files[bg_index])
    
    background = cv2.imread(bg)
    rows_b, cols_b, channels_b = background.shape
    
    return background, rows_b, cols_b, channels_b 

# 3. Combine image with alpha channel and background

In [4]:
def semi_artificial_data_creation(bg_path, path):
    """
    goes through all the images in the folder
    randomly picks a background image
    pastes the ingredient image with transparent background on the background image
    """
    
    for root, dirs, files in os.walk(path):
        for dir in dirs:
            parent_folder, _ = os.path.split(path)
            folder_art_name = f'{dir}_semi_art'
            folder_art_path = os.path.join(root, folder_art_name)
            os.makedirs(folder_art_path)

        files = [f for f in files if not f[0] == '.' and f.find('bg') == -1] 
        for file in files: 
            if file.endswith(".png"):
                img_path = os.path.join(root, file)
                ingredient = Image.open(img_path)

                # background generation
                background = load_bg_random_art(bg_path)[0]
                background = cv2.cvtColor(background, cv2.COLOR_BGR2RGB)
                background = Image.fromarray(background)
                bg = background.resize((300, 300)).copy()

                # resize ingredient image to be the same size as the background
                ingredient_res = ingredient.resize((300,300))

                # combine ingredient image with transparent background and random background
                bg.paste(ingredient_res, (0,0) , ingredient_res)

                # save artificially created image in a new sub-folder
                img_art_name = os.path.splitext(file)[0]
                
                current_ingr_path, _ = os.path.split(img_path)
                _, current_ingr_folder = os.path.split(current_ingr_path)
                parent_folder, _ = os.path.split(current_ingr_path)
                
                folder_art_name = f'{current_ingr_folder}_semi_art'
                img_art_path = os.path.join(parent_folder, folder_art_name, f'{img_art_name}_semi_art.jpg')
                
                bg.save(img_art_path)

In [5]:
path = '../scraping/scraped_images'
bg_path = './background'

In [6]:
semi_artificial_data_creation(bg_path, path)

In [7]:
img = Image.open('../scraping/scraped_images/carrot_semi_art/image00004_semi_art.jpg')

In [8]:
img.show()